Face web scraping do ferecast de DXY

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL da página de previsão do U.S. Dollar Index
url = "https://www.forecasts.org/dollar-forecast.htm"

# Função para obter os dados do índice
def get_dxy_data():
    # Fazer a requisição HTTP para a página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Criar o objeto BeautifulSoup para processar o HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID (caso tenha um identificador)
    table = soup.find("table", {"id": "forecast"})
    
    # Extrair todas as linhas da tabela
    rows = table.find_all("tr")[1:]  # Ignora o cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []
    
    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Convertendo a string para uma data válida
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_dxy = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # --- Capturar a data de atualização ---
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_dxy, updated_date

# Chamar a função para obter os dados
df_dxy, last_updated = get_dxy_data()

# Exibir os dados
display(df_dxy)
display(df_dxy.columns)
print(f"\nÚltima atualização: {last_updated}")

,Month,Date,Forecast Value,Avg Error
0,Apr 2025,2025-04-01,100.725,0.00
1,May 2025,2025-05-01,101.280,0.38
2,Jun 2025,2025-06-01,103.450,0.46
3,Jul 2025,2025-07-01,105.310,0.50
4,Aug 2025,2025-08-01,104.200,0.53
5,Sep 2025,2025-09-01,107.420,0.56
6,Oct 2025,2025-10-01,108.190,0.58
7,Nov 2025,2025-11-01,106.770,0.59
8,Dec 2025,2025-12-01,103.830,0.61


Index(['Month', 'Date', 'Forecast Value', 'Avg Error'], dtype='object')


Última atualização: 2025-05-02 00:00:00


Bono 10 y

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os

# URL da página de previsão dos Treasury Yields
url = "https://www.forecasts.org/10yrT.htm"

# Arquivo para armazenar a última atualização
UPDATE_FILE = "last_update.txt"

# Função para obter os dados da tabela
def get_treasury_data():
    # Fazer requisição HTTP para obter o conteúdo da página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição

    # Processar o HTML com BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Localizar a tabela pelo ID "forecast"
    table = soup.find("table", {"id": "forecast"})
    rows = table.find_all("tr")[1:]  # Ignorar cabeçalho

    # Criar lista para armazenar os dados extraídos
    data_list = []

    for row in rows:
        cols = row.find_all("td")
        month = cols[1].text.strip()
        date = pd.to_datetime(month)  # Converter para formato datetime
        forecast_value = float(cols[2].text.strip())
        avg_error = float(cols[3].text.replace("±", "").strip())

        data_list.append([month, date, forecast_value, avg_error])

    # Criar DataFrame
    df_treasury = pd.DataFrame(data_list, columns=["Month", "Date", "Forecast Value", "Avg Error"])

    # Capturar a data de atualização
    updated_text = soup.find("time").text.strip()
    updated_date = pd.to_datetime(updated_text.replace("Updated: ", ""))  # Converter para formato datetime

    return df_treasury, updated_date

# Função para verificar e salvar atualização
def check_for_update():
    df_treasury, last_updated = get_treasury_data()

    # Verificar se há um arquivo de última atualização salvo
    if os.path.exists(UPDATE_FILE):
        with open(UPDATE_FILE, "r") as file:
            last_saved_date = pd.to_datetime(file.read().strip())

        # Se houver uma nova atualização, salvar novo CSV e atualizar o arquivo de controle
        if last_updated > last_saved_date:
            print(f"Nova atualização encontrada em {last_updated}! Salvando dados...")
            df_treasury.to_csv("treasury_forecast.csv", index=False)

            with open(UPDATE_FILE, "w") as file:
                file.write(str(last_updated))
        else:
            print("Nenhuma atualização nova encontrada.")
    else:
        # Primeira execução - salvar a data
        with open(UPDATE_FILE, "w") as file:
            file.write(str(last_updated))
        print("Primeira execução do script. Criando CSV inicial.")
        df_treasury.to_csv("treasury_forecast.csv", index=False)

# Executar a verificação automática
check_for_update()

Nenhuma atualização nova encontrada.
